Importing libraries needed to fetch web-site content and create data file respectively.

In [1]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-cp37-none-any.whl size=1278 sha256=6f36459859975fbaf8a68aaae2066cece898383d339e0f75c249a9fc8ee90abe
  Stored in directory: C:\Users\Mykhaylo_Manukyan\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
import pandas as pd
import numpy as np
import bs4
import re
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

Creating starting URL pointing to search result page with open vacancies on headhunter.com web site using "data science" as search term.

In [ ]:
keyWord = 'data science'
keyWord = keyWord.replace(' ', '%20')
urlorig='https://www.headhunter.com/jobs?Keywords='+keyWord+'&Radius=150&PageNumber=1&OrderBy=Relevance'
urlorig

This piece of code should go through search results and extract links which pointing to full list of jobs per company. This is specific of headhunter.com web site. Since list of jobs per company has also multiple number of pages - code will recursively call method to fetch all next pages till end of list.

In [ ]:
def get_pages(soup_param, res_list):
    pages = soup_param.findAll("a", class_ = "searchnext")
    if len(pages) > 0:
        link = pages[0]
        link_esc = link["href"].replace(' ', '%20')
        print(link["href"])
        Jb_page = uReq("https://www.headhunter.com" + link_esc)
        Jb_page_html = Jb_page.read()
        Jb_page_soup = soup(Jb_page_html, 'html.parser')
        Jb_page.close()
        res_list.append(Jb_page_soup)
        get_pages(Jb_page_soup, res_list)


my_url = urlorig
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,'html.parser')

jb_all = []


links_all_jobs = page_soup.findAll("a", class_ = "see_all_jobs_link")
for link in links_all_jobs:
    Jb_all = uReq("https://www.headhunter.com" + link["href"])
#     print(link["href"])
    Jb_all_html = Jb_all.read()
    Jb_all.close()
    Jb_all_soup = soup(Jb_all_html, "html.parser")
    jb_all.append(Jb_all_soup)
    
all_pages=[]
for item in jb_all:
    get_pages(item, all_pages)

jb_all.extend(all_pages)


Cells below were used for try and error method of parsing single job details page. I used this to speed up trouble-shooting and finetune code, which will be later executed in cycles.

In [ ]:
#single vacancy page
# # my_url1 = "https://www.headhunter.com/jobs/Data-Analytics-Manager/J3M2FR6FRKHG9GZFN1W"
# my_url1 = "https://www.headhunter.com/jobs/Senior-Data-Scientist/J3N61M605D08SG4PV0C"
# uClient1 = uReq(my_url1)
# page_html1 = uClient1.read()
# uClient1.close()
# page_soup1 = soup(page_html1,'html.parser')

# print(page_soup1)



In [ ]:
# content = page_soup1.findAll("div", class_="content")
# print(len(content))
# for i in content:
#     print(i.text)
#     refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
#     print(refID.group()[8:])

# job_text = page_soup1.findAll("div", id=("description-container-premium", "description-container-standard"))
# item = job_text[0]
# contents = item.select("div.content")
# # print(contents)

# refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
# if refID != None:
#     print(refID.group()[8:])

# descr=''
# for content in contents:

    
#     descr+= content.text
    
#     for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
#         if i.text == 'Location ':
#             print(i.next_sibling.string.next_sibling.text)
#         if i.text == 'Employment Type ':
#             print(i.next_sibling.string.next_sibling.text)
#         if i.text == 'Pay Type ':
#             print(i.next_sibling.string.next_sibling.text)
#         if i.text == 'Pay Rate ':
#             print(i.next_sibling.string.next_sibling.text)
#         if i.text == 'Store Type ':
#             print(i.next_sibling.string.next_sibling.text)
#         if i.text == 'Other Compensation: ':
#             print(i.next_sibling.string.next_sibling.text)
# # print(descr)

# if("statistic" in descr.lower()):
#     print('1')
# else:
#     print('0')

Next cell is a parser itself for headhunter.com web site. Search result itself are presented as short list (3 jobs per company) with a link leads to all jobs per company. Hence parsing happens in two steps - first parse all jobs from short list. Next step, parse all details from jobs under relative web pages (including sub-pages of each full job list per company). Code requires refactoring to exclude copy-paste, but currently it works and fetch all jobs.

In [ ]:
titles = []
company_names = []
location_names = []
compensation = []
job_descriptions = []
job_ids = []
employment_types = []
pay_types=[]
pay_rates=[]
store_types=[]
other_compensations=[]
refs=[]
skills = {}
skills['Python'] = []
skills['R '] = []
skills['Matlab'] = []
skills['Excel'] = []
skills['SAS'] = []
skills['SQL'] = []
skills['SPSS'] = []
skills['Hadoop'] = []
skills['Spark'] = []
skills['Algebra'] = []
skills['Statistic'] = []
skills['Mathematics'] = []
skills['Reinforcement_Learning'] = []
skills['Deep_Learning'] = []
skills['Machine_Learning'] = []
skills['Data_Visualization'] = []
skills['Keras'] = []
skills['TensorFlow'] = []
skills['Artificial_Intelligence'] = []
skills['Communications'] = []
skills['Team_Player'] = []
skills['Empathy'] = []
skills['Emotional_Intellect'] = []
skills['Self_Improvement'] = []
skills['Presentation'] = []
skills['Critical_Thinking'] = []
skills['Business_Understanding'] = []

# parsing short list of vacancies

# extract job_title
jobs = page_soup.findAll("div", class_=re.compile("searchresult clearfix"))
for job in jobs:
    try:
        titles.append(job["title"])
    except:
        titles.append("NA")

# print(titles)

# extract company_name
companies = page_soup.findAll("span", class_="resultcompany_mobile clearfix" )
for company in companies:
    try:
        company_names.append(company.text.strip())
    except:
        company_names.append("NA")
        
# print(company_names)

# extract location       
locations = page_soup.findAll("span", class_="result-right-one")
for location in locations:
    try:
        location_names.append(location.text)
    except:
        location_names.append("NA")
        
# print(location_names)

# extract compensation       
payments = page_soup.findAll("span", class_="result-right-two non-mobile")
for payment in payments:
    try:
        compensation.append(payment.text)
    except:
        compensation.append("NA")
        
# print(compensation)

#extract job_id and full description
links = page_soup.findAll("a", class_ = "result-left")
ind=0
for link in links:
    ind+=1
    print(ind, link["href"])
    Jb = uReq("https://www.headhunter.com" + link["href"])
    Jb_html = Jb.read()
    Jb.close()
    Jb_soup = soup(Jb_html, "html.parser")
    
    try:
        refs.append("https://www.headhunter.com" + link["href"])
    except:
        refs.append("NA")
    
    job_text = Jb_soup.findAll("div", id=("description-container-premium", "description-container-standard"))
    item = job_text[0]
    contents = item.select("div.content")

    refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
    try:
        job_ids.append(refID.group()[8:])
    except:
        job_ids.append("NA")
    
    descr=''
    other_comp_flag=False
    for content in contents:
        
        descr+= content.text
        
    
    
        for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
    #         if i.text == 'Location ':
    #             print(i.next_sibling.string.next_sibling.text)

    #         other_compensations.append("NA")

            if i.text == 'Employment Type ':
                try:
                    employment_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    employment_types.append("NA")
            if i.text == 'Pay Type ':
                try:
                    pay_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    pay_types.append("NA")
            if i.text == 'Pay Rate ':
                try:
                    pay_rates.append(i.next_sibling.string.next_sibling.text)
                except:
                    pay_rates.append("NA")
            if i.text == 'Store Type ':
                try:
                    store_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    store_types.append("NA")
            if i.text == 'Other Compensation: ':
                other_comp_flag=True
                try:
                    other_compensations.append(i.next_sibling.string.next_sibling.text)
                except:
                    other_compensations.append("NA")
                
    try:
        job_descriptions.append(str(re.sub('(\n|\r)', '',descr)))
    except:
        job_descriptions.append("NA")
        
# looking for skills - Excell
    if("excel" in descr.lower()):
        skills['Excel'].append('1')
    else:
        skills['Excel'].append('0')
# looking for skills - Python    
    if("python" in descr.lower()):
        skills['Python'].append('1')
    else:
        skills['Python'].append('0')
# looking for skills - R
    if("R " in descr):
        skills['R '].append('1')
    else:
        skills['R '].append('0')
# looking for skills - Matlab
    if("matlab" in descr.lower()):
        skills['Matlab'].append('1')
    else:
        skills['Matlab'].append('0')
# looking for skills - SAS
    if("sas" in descr.lower()):
        skills['SAS'].append('1')
    else:
        skills['SAS'].append('0')
# looking for skills - SQL
    if("sql" in descr.lower()):
        skills['SQL'].append('1')
    else:
        skills['SQL'].append('0')
# looking for skills - SPSS
    if("spss" in descr.lower()):
        skills['SPSS'].append('1')
    else:
        skills['SPSS'].append('0')
# looking for skills - Hadoop
    if("hadoop" in descr.lower()):
        skills['Hadoop'].append('1')
    else:
        skills['Hadoop'].append('0')
# looking for skills - Spark
    if("spark" in descr.lower()):
        skills['Spark'].append('1')
    else:
        skills['Spark'].append('0')
# looking for skills - Algebra
    if("algebra" in descr.lower()):
        skills['Algebra'].append('1')
    else:
        skills['Algebra'].append('0')
# looking for skills - Statistic
    if("statistic" in descr.lower()):
        skills['Statistic'].append('1')
    else:
        skills['Statistic'].append('0')
# looking for skills - Mathematics
    if("math" in descr.lower()):
        skills['Mathematics'].append('1')
    else:
        skills['Mathematics'].append('0')
# looking for skills - Reinforcement Learning
    if("reinforcement learning" in descr.lower()):
        skills['Reinforcement_Learning'].append('1')
    else:
        skills['Reinforcement_Learning'].append('0')
# looking for skills - Deep Learning    
    if("deep learning" in descr.lower()):
        skills['Deep_Learning'].append('1')
    else:
        skills['Deep_Learning'].append('0')
# looking for skills - Machine Learning                                   
    if("machine learning" in descr.lower()):
        skills['Machine_Learning'].append('1')
    else:
        skills['Machine_Learning'].append('0')
# looking for skills - Data Visualization
    if("visualization" in descr.lower()):
        skills['Data_Visualization'].append('1')
    else:
        skills['Data_Visualization'].append('0')
# looking for skills - Keras
    if("keras" in descr.lower()):
        skills['Keras'].append('1')
    else:
        skills['Keras'].append('0')
# looking for skills - TensorFlow
    if("tensorflow" in descr.lower()):
        skills['TensorFlow'].append('1')
    else:
        skills['TensorFlow'].append('0')
# looking for skills - Artificial Intelligence
    if("artificial intelligence" in descr.lower()):
        skills['Artificial_Intelligence'].append('1')
    else:
        skills['Artificial_Intelligence'].append('0')
# looking for skills - Communications
    if("communication" in descr.lower()):
        skills['Communications'].append('1')
    else:
        skills['Communications'].append('0')
# looking for skills - Team Player
    if("team work" in descr.lower() or "team spirit" in descr.lower() or "team member" in descr.lower() 
       or "team player" in descr.lower()):
        skills['Team_Player'].append('1')
    else:
        skills['Team_Player'].append('0')
# looking for skills - Empathy
    if("empathy" in descr.lower()):
        skills['Empathy'].append('1')
    else:
        skills['Empathy'].append('0')
# looking for skills - Emotional Intellect
    if("emotional intellect" in descr.lower()):
        skills['Emotional_Intellect'].append('1')
    else:
        skills['Emotional_Intellect'].append('0')
# looking for skills - Self-Improvement
    if("self-improvement" in descr.lower() or "personal development" in descr.lower()):
        skills['Self_Improvement'].append('1')
    else:
        skills['Self_Improvement'].append('0')
# looking for skills - Presentation
    if("presentation" in descr.lower()):
        skills['Presentation'].append('1')
    else:
        skills['Presentation'].append('0')
# looking for skills - Critical Thinking
    if("critical thinking" in descr.lower()):
        skills['Critical_Thinking'].append('1')
    else:
        skills['Critical_Thinking'].append('0')
# looking for skills - Business Understanding
    if("business understanding" in descr.lower() or "understand business" in descr.lower()):
        skills['Business_Understanding'].append('1')
    else:
        skills['Business_Understanding'].append('0')
    
    if other_comp_flag==False:
        other_compensations.append("NA")
    
#     try:
#         content = Jb_soup.find("div", class_="content")
#         refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
#         job_ids.append(refID.group()[8:])
#     except:
#         job_ids.append("NA")    
        
#     try:
#         job_descriptions.append(str(re.sub('(\n|\r)', '',content.text)))
#     except:
#         job_descriptions.append("NA")
        
        
# print(job_descriptions)

#extract all jobsfor company with many vacancies
for item in jb_all:
    
    # extract job_title
    jobs = item.findAll("div", class_=re.compile("searchresult clearfix"))
    for job in jobs:
        try:
            titles.append(job["title"])
        except:
            titles.append("NA")
            
    # extract company_name
    companies = item.findAll("span", class_="resultcompany_mobile clearfix" )
    for company in companies:
        try:
            company_names.append(company.text.strip())
        except:
            company_names.append("NA")

    # extract location       
    locations = item.findAll("span", class_="result-right-one")
    for location in locations:
        try:
            location_names.append(location.text)
        except:
            location_names.append("NA")

    # extract compensation       
    payments = item.findAll("span", class_="result-right-two non-mobile")
    for payment in payments:
        try:
            compensation.append(payment.text)
        except:
            compensation.append("NA")

    #extract job description and all information rom it
    links = item.findAll("a", class_ = "result-left")
    for link in links:
        ind+=1
        print(ind, link["href"])
        Jb = uReq("https://www.headhunter.com" + link["href"])
        Jb_html = Jb.read()
        Jb.close()
        Jb_soup = soup(Jb_html, "html.parser")
        
        try:
            refs.append("https://www.headhunter.com" + link["href"])
        except:
            refs.append("NA")

        job_text = Jb_soup.findAll("div", id=("description-container-premium", "description-container-standard"))
        item = job_text[0]
        contents = item.select("div.content")

        refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
        try:
            job_ids.append(refID.group()[8:])
        except:
            job_ids.append("NA")

        descr=''
        other_comp_flag=False
        for content in contents:
        
            descr+= content.text

            for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
        #         if i.text == 'Location ':
        #             print(i.next_sibling.string.next_sibling.text)

        #         other_compensations.append("NA")

                if i.text == 'Employment Type ':
                    try:
                        employment_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        employment_types.append("NA")
                if i.text == 'Pay Type ':
                    try:
                        pay_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        pay_types.append("NA")
                if i.text == 'Pay Rate ':
                    try:
                        pay_rates.append(i.next_sibling.string.next_sibling.text)
                    except:
                        pay_rates.append("NA")
                if i.text == 'Store Type ':
                    try:
                        store_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        store_types.append("NA")
                if i.text == 'Other Compensation: ':
                    other_comp_flag=True
                    try:
                        other_compensations.append(i.next_sibling.string.next_sibling.text)
                    except:
                        other_compensations.append("NA")

        try:
            job_descriptions.append(str(re.sub('(\n|\r)', '',descr)))
        except:
            job_descriptions.append("NA")
            
    # looking for skills - Excell
        if("excel" in descr.lower()):
            skills['Excel'].append('1')
        else:
            skills['Excel'].append('0')
    # looking for skills - Python    
        if("python" in descr.lower()):
            skills['Python'].append('1')
        else:
            skills['Python'].append('0')
    # looking for skills - R
        if("R " in descr):
            skills['R '].append('1')
        else:
            skills['R '].append('0')
    # looking for skills - Matlab
        if("matlab" in descr.lower()):
            skills['Matlab'].append('1')
        else:
            skills['Matlab'].append('0')
    # looking for skills - SAS
        if("sas" in descr.lower()):
            skills['SAS'].append('1')
        else:
            skills['SAS'].append('0')
    # looking for skills - SQL
        if("sql" in descr.lower()):
            skills['SQL'].append('1')
        else:
            skills['SQL'].append('0')
    # looking for skills - SPSS
        if("spss" in descr.lower()):
            skills['SPSS'].append('1')
        else:
            skills['SPSS'].append('0')
    # looking for skills - Hadoop
        if("hadoop" in descr.lower()):
            skills['Hadoop'].append('1')
        else:
            skills['Hadoop'].append('0')
    # looking for skills - Spark
        if("spark" in descr.lower()):
            skills['Spark'].append('1')
        else:
            skills['Spark'].append('0')
    # looking for skills - Algebra
        if("algebra" in descr.lower()):
            skills['Algebra'].append('1')
        else:
            skills['Algebra'].append('0')
    # looking for skills - Statistic
        if("statistic" in descr.lower()):
            skills['Statistic'].append('1')
        else:
            skills['Statistic'].append('0')
    # looking for skills - Mathematics
        if("math" in descr.lower()):
            skills['Mathematics'].append('1')
        else:
            skills['Mathematics'].append('0')
    # looking for skills - Reinforcement Learning
        if("reinforcement learning" in descr.lower()):
            skills['Reinforcement_Learning'].append('1')
        else:
            skills['Reinforcement_Learning'].append('0')
    # looking for skills - Deep Learning    
        if("deep learning" in descr.lower()):
            skills['Deep_Learning'].append('1')
        else:
            skills['Deep_Learning'].append('0')
    # looking for skills - Machine Learning                                   
        if("machine learning" in descr.lower()):
            skills['Machine_Learning'].append('1')
        else:
            skills['Machine_Learning'].append('0')
    # looking for skills - Data Visualization
        if("visualization" in descr.lower()):
            skills['Data_Visualization'].append('1')
        else:
            skills['Data_Visualization'].append('0')
    # looking for skills - Keras
        if("keras" in descr.lower()):
            skills['Keras'].append('1')
        else:
            skills['Keras'].append('0')
    # looking for skills - TensorFlow
        if("tensorflow" in descr.lower()):
            skills['TensorFlow'].append('1')
        else:
            skills['TensorFlow'].append('0')
    # looking for skills - Artificial Intelligence
        if("artificial intelligence" in descr.lower()):
            skills['Artificial_Intelligence'].append('1')
        else:
            skills['Artificial_Intelligence'].append('0')
    # looking for skills - Communications
        if("communication" in descr.lower()):
            skills['Communications'].append('1')
        else:
            skills['Communications'].append('0')
    # looking for skills - Team Player
        if("team work" in descr.lower() or "team spirit" in descr.lower() or "team member" in descr.lower() 
           or "team player" in descr.lower()):
            skills['Team_Player'].append('1')
        else:
            skills['Team_Player'].append('0')
    # looking for skills - Empathy
        if("empathy" in descr.lower()):
            skills['Empathy'].append('1')
        else:
            skills['Empathy'].append('0')
    # looking for skills - Emotional Intellect
        if("emotional intellect" in descr.lower()):
            skills['Emotional_Intellect'].append('1')
        else:
            skills['Emotional_Intellect'].append('0')
    # looking for skills - Self-Improvement
        if("self-improvement" in descr.lower() or "personal development" in descr.lower()):
            skills['Self_Improvement'].append('1')
        else:
            skills['Self_Improvement'].append('0')
    # looking for skills - Presentation
        if("presentation" in descr.lower()):
            skills['Presentation'].append('1')
        else:
            skills['Presentation'].append('0')
    # looking for skills - Critical Thinking
        if("critical thinking" in descr.lower()):
            skills['Critical_Thinking'].append('1')
        else:
            skills['Critical_Thinking'].append('0')
    # looking for skills - Business Understanding
        if("business understanding" in descr.lower() or "understand business" in descr.lower()):
            skills['Business_Understanding'].append('1')
        else:
            skills['Business_Understanding'].append('0')

        if other_comp_flag==False:
            other_compensations.append("NA")

Create pandas dataframe from list and dictionaries filled during parsing

In [ ]:
d = {'Job_Title':titles,'Company_Name':company_names, 'Location':location_names, 'Compensation': compensation, 
     'Job_ID':job_ids,'Job_Description':job_descriptions, 'Employment_Type':employment_types, 'Pay_Type':pay_types,
     'Pay_Rate':pay_rates, 'Store_Type':store_types, 'Other_Compensation':other_compensations, 'Link': refs}

df1 = pd.DataFrame(d)
df2 = pd.DataFrame(skills)
frames = [df1, df2]
results = pd.concat(frames, axis = 1)
results.head()

Since we parse data from short list and full list - there will be duplicates for sure. Hence removing duplicates using pandas dataframe method.

In [ ]:
print(len(results))
result_unique = results.drop_duplicates()
print(len(result_unique))

Saving dataframe into csv file

In [ ]:
result_unique.to_csv('Headhunters_job_vacancies.csv')